In [1]:
import json
import requests
import pandas as pd

#import keras

In [2]:
info = requests.get('https://api.hackupm2025.workers.dev/api/v1/patients/train')
info = json.loads(info.text)['pagination']

In [3]:
n_pages = info['totalPages']
data = []
for page in range(n_pages):
    r = requests.get(f'https://api.hackupm2025.workers.dev/api/v1/patients/train?page={page+1}')
    page_data = json.loads(r.text)
    data.extend(page_data['data'])

In [4]:
import pandas as pd

df = pd.DataFrame(data)
df

,patient_id,has_diabetes,medical_note
0,82555,0,The patient is a 16-year-old female with a BMI...
1,92299,0,"Subjective: The patient, a 15-year-old female,..."
2,18725,0,The patient is a 54-year-old male with a BMI o...
3,52208,1,The patient is a 54-year-old female with a BMI...
4,2640,0,**Subjective:** \nThe patient is a 29-year-ol...
...,...,...,...
2995,41870,0,Subjective: The patient is a 36-year-old femal...
2996,10295,0,The patient is a four-year-old female with a B...
2997,28610,0,The patient is a 66-year-old female with a BMI...
2998,14177,0,"Subjective: The patient, a 75-year-old male wi..."


In [5]:
df.has_diabetes

0       0
1       0
2       0
3       1
4       0
       ..
2995    0
2996    0
2997    0
2998    0
2999    1
Name: has_diabetes, Length: 3000, dtype: int64

In [6]:
df.to_csv("./data.csv")

In [7]:
df = pd.read_csv("data.csv")

In [8]:
df.iloc[0]

Unnamed: 0                                                      0
patient_id                                                  82555
has_diabetes                                                    0
medical_note    The patient is a 16-year-old female with a BMI...
Name: 0, dtype: object

In [23]:
import re

def search_age(note: str) -> int | None:
    AGE_RE = re.compile(r'(\d{1,3})-year-old', re.I)
    match = re.search(AGE_RE, note)
    return int(match.group(1)) if match else None

def search_gender(note: str) -> bool | None:
    GENDER_RE_MALE = re.compile(r'\b(male)\b', re.I)
    GENDER_RE_FEMALE = re.compile(r'\b(female)\b', re.I)
    matchmale = re.search(GENDER_RE_MALE, note)
    matchfemale = re.search(GENDER_RE_FEMALE, note)
    if matchmale:
        return 0
    if matchfemale:
        return 1
    return None

def search_smoker(note: str) -> bool | None:
    if not isinstance(note, str):
        return None
    note_lower = note.lower()
    if re.search(r'non[-\s]?smok', note_lower):
        return 0
    elif re.search(r'\bsmok(er|es|ing)?\b', note_lower):
        return 1
    return None

def search_bmi(note: str) -> float | None:
    if not isinstance(note, str):
        return None
    BMI_RE = re.compile(r'\bBMI\b[^0-9]*([0-9]+(?:\.[0-9]+)?)', re.I)
    match = re.search(BMI_RE, note)
    return float(match.group(1)) if match else None

def search_glucose(note: str) -> int | None:
    if not isinstance(note, str):
        return None
    note_lower = note.lower()

    # Patrón mejorado para glucosa
    GLUCOSE_RE = re.compile(
        r'\b(?:fasting|blood)?\s*glucose[^0-9]{0,10}(\d{2,3})(?:\s*mg/?dL)?\b',
        re.I
    )

    match = re.search(GLUCOSE_RE, note_lower)
    if match:
        return int(match.group(1))
    return None



def search_HbA1c(note: str) -> float | None:
    if not isinstance(note, str):
        return None
    note_lower = note.lower()
    # Patrón robusto que cubre múltiples formas de escribir HbA1c
    HBA1C_RE = re.compile(
        r'\b(?:hba1c|a1c|hemoglobin\s*a1c)\b[^\d]{0,10}(\d+(?:\.\d+)?)\s*(?:%|percent)?',
        re.I
    )

    match = re.search(HBA1C_RE, note_lower)
    if match:
        return float(match.group(1))
    return None


def search_hypertension(note: str) -> bool | None:
    sentences = re.split(r'(?<=[.])', note.strip())
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    for i, oracion in enumerate(sentences, 1):
        contains_hypertension_or_heart_disease = "hypertension or heart disease" in oracion.lower()
        if (contains_hypertension_or_heart_disease):
            hypertension_or_HD_RE = re.compile(r'\b(no|not)\b[^.?!]*\bhypertension\s+or\s+heart\s+disease\b', re.I)
            match = re.search(hypertension_or_HD_RE, note)
            if(match):
                return 0
            else:
                return 1
        contains_hypertension = "hypertension" in oracion.lower()
        if(contains_hypertension):
            hypertension_RE = re.compile(r'\b(no|not)\b[^.?!]*\bhypertension\b', re.I)
            match = re.search(hypertension_RE, note)
            if(match):
                return 0
            else:
                return 1
    return None

def search_heart_disease(note: str) -> bool | None:
    sentences = re.split(r'(?<=[.])', note.strip())
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    for i, oracion in enumerate(sentences, 1):
        contains_hypertension_or_heart_disease = "hypertension or heart disease" in oracion.lower()
        if (contains_hypertension_or_heart_disease):
            hypertension_or_HD_RE = re.compile(r'\b(no|not)\b[^.?!]*\bhypertension\s+or\s+heart\s+disease\b', re.I)
            match = re.search(hypertension_or_HD_RE, note)
            if(match):
                return 0
            else:
                return 1
        contains_heart_disease = "heart disease" in oracion.lower()
        if(contains_heart_disease):
            heart_disease_RE = re.compile(r'\b(no|not)\b[^.?!]*\bheart\s+disease\b', re.I)
            match = re.search(heart_disease_RE, note)
            if(match):
                return 0
            else:
                return 1
    return None

def search_HbA1c_qualitative(note: str) -> int | None:
    if not isinstance(note, str):
        return None

    text = note.lower()

    normal_patterns = [
        r'\bhba1c\b[^.]{0,20}\b(normal|within\s+normal\s+limits|good\s+control)\b',
        r'\b(normal|within\s+normal\s+limits|good\s+control)\b[^.]{0,20}\bhba1c\b'
    ]

    high_patterns = [
        r'\bhba1c\b[^.]{0,20}\b(elevated|high|raised|abnormal|poor\s+control)\b',
        r'\b(elevated|high|raised|abnormal|poor\s+control)\b[^.]{0,20}\bhba1c\b'
    ]

    for p in normal_patterns:
        if re.search(p, text):
            return 0

    for p in high_patterns:
        if re.search(p, text):
            return 1

    return None

def search_glucose_qualitative(note: str) -> int | None:
    if not isinstance(note, str):
        return None

    text = note.lower()

    # Patrones de normalidad
    normal_patterns = [
        r'\b(glucose|blood\s+glucose|random\s+glucose)\b[^.]{0,25}\b(normal|within\s+normal\s+limits|good\s+control)\b',
        r'\b(normal|within\s+normal\s+limits|good\s+control)\b[^.]{0,25}\b(glucose|blood\s+glucose|random\s+glucose)\b'
    ]

    # Patrones de elevación
    high_patterns = [
        r'\b(glucose|blood\s+glucose|random\s+glucose)\b[^.]{0,25}\b(elevated|high|raised|abnormal|increased|hyperglycemia)\b',
        r'\b(elevated|high|raised|abnormal|increased|hyperglycemia)\b[^.]{0,25}\b(glucose|blood\s+glucose|random\s+glucose)\b'
    ]

    for p in normal_patterns:
        if re.search(p, text):
            return 0

    for p in high_patterns:
        if re.search(p, text):
            return 1

    return None



In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize



def preprocess(row: pd.Series) -> dict:
    return {
        "patient_id": row.get("patient_id"),
        "has_diabetes": row.get("has_diabetes"),
        "age": search_age(row['medical_note']),
        "gender": search_gender(row['medical_note']),
        "hypertension": search_hypertension(row['medical_note']),
        "heart_disease": search_heart_disease(row['medical_note']),
        "smoker": search_smoker(row['medical_note']),
        "bmi": search_bmi(row['medical_note']),
        "glucose": search_glucose(row['medical_note']),
        "hba1c": search_HbA1c(row['medical_note']),
        "hba1c_cual": search_HbA1c_qualitative(row['medical_note']),
        "glucose_cual": search_glucose_qualitative(row['medical_note']),
        
    }  
processed_df = df.apply(preprocess, axis=1, result_type='expand')
processed_df




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\halva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,patient_id,has_diabetes,age,gender,hypertension,heart_disease,smoker,bmi,glucose,hba1c,hba1c_cual,glucose_cual
0,82555.0,0.0,16.0,1.0,0.0,0.0,0.0,21.49,NaN,6.2,NaN,0.0
1,92299.0,0.0,15.0,1.0,0.0,0.0,0.0,33.62,158.0,NaN,0.0,0.0
2,18725.0,0.0,54.0,0.0,0.0,0.0,1.0,21.46,145.0,NaN,0.0,NaN
3,52208.0,1.0,54.0,1.0,0.0,0.0,0.0,39.12,NaN,NaN,1.0,1.0
4,2640.0,0.0,29.0,1.0,0.0,0.0,1.0,38.25,126.0,3.5,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,41870.0,0.0,36.0,1.0,0.0,0.0,1.0,24.47,NaN,NaN,1.0,1.0
2996,10295.0,0.0,NaN,1.0,0.0,0.0,0.0,15.96,90.0,6.1,NaN,NaN
2997,28610.0,0.0,66.0,1.0,0.0,0.0,0.0,26.70,140.0,5.0,NaN,0.0
2998,14177.0,0.0,75.0,0.0,0.0,0.0,1.0,25.54,NaN,6.1,NaN,0.0
